In [1]:
import asyncio

from joblib import Parallel, delayed

import requests as req
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select

import time
import multiprocessing as mp

import pandas as pd
pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

from tqdm.notebook import tqdm

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH=r'Chromedriver\chromedriver'

import warnings
warnings.filterwarnings('ignore')

# Stats

In [2]:
stats_original = pd.read_csv(r'stats_first_clean.csv')
stats = stats_original.copy()
stats.head()

,Unnamed: 0,name,username,profile,user_id,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,"Engine, Komodo",komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0


In [3]:
stats = stats.drop(['profile', 'user_id'], axis=1)
stats

,Unnamed: 0,name,username,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,"Engine, Komodo",komodochess,GM,3411.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",vojtechplat,GM,2950.0,Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",playmagnus,GM,2882.0,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",magnuscarlsen,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",fabianocaruana,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,"Zvonko, Stanojoski",zona1,GM,NaN,North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,2102.0,2083.0,13.0,1.0,7.0
1504,1504,"Gurevich, Dmitry",zq1,GM,NaN,United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,2400.0,2594.0,496.0,47.0,402.0
1505,1505,"Zubov, Alexander",zubov_alexander,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN
1506,1506,"Zubov, Alexander",zubov_on_youtube,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN


# Df

In [4]:
df_original = pd.read_csv(r'top_100_scraping.csv')
df = df_original.copy()
df.head()

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Ding, Liren",760 (33.48 %),413 (47.4 %),434 (19.12 %)
4,"Firouzja, Alireza",1484 (52.68 %),223 (23.82 %),662 (23.5 %)


# Table

In [5]:
table_original = pd.read_csv(r'rating_top_100.csv')
table = table_original.copy()
table.head()

,#,Name,Classic,Rapid,Blitz,Age
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
3,4,"Ding, Liren",2780.0,2830.0,2787 i,30
4,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20


In [6]:
table.columns = ['#', 'name', 'Classic', 'Rapid', 'Blitz', 'Age']
table

,#,name,Classic,Rapid,Blitz,Age
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
3,4,"Ding, Liren",2780.0,2830.0,2787 i,30
4,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20
...,...,...,...,...,...,...
95,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46
96,97,"Saric, Ivan",2647.5,2631.0,2586.0,32
97,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42
98,99,"Liang, Awonder",2645.6,2518.4,2538.0,20


### Merge

In [7]:
table = pd.merge(table, df, on='name', how='left')
table

,#,name,Classic,Rapid,Blitz,Age,Wins,Draws,Losses
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32,2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31,2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,NaN,NaN,NaN
3,4,"Ding, Liren",2780.0,2830.0,2787 i,30,760 (33.48 %),413 (47.4 %),434 (19.12 %)
4,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20,1484 (52.68 %),223 (23.82 %),662 (23.5 %)
...,...,...,...,...,...,...,...,...,...
95,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46,1181 (43.18 %),None (34.52 %),610 (22.3 %)
96,97,"Saric, Ivan",2647.5,2631.0,2586.0,32,1153 (43.31 %),None (36.03 %),550 (20.66 %)
97,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42,NaN,NaN,NaN
98,99,"Liang, Awonder",2645.6,2518.4,2538.0,20,NaN,NaN,NaN


In [8]:
table = pd.merge(table, stats, on='name', how='left')
table

,#,name,Classic,Rapid,Blitz,Age,Wins,Draws,Losses,Unnamed: 0,username,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32,2879 (48.53 %),718 (35.27 %),961 (16.2 %),3.0,magnuscarlsen,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31,2025 (42.75 %),547 (34.49 %),1078 (22.76 %),4.0,fabianocaruana,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,NaN,NaN,NaN,6.0,hikaru,GM,2814.0,United States,2781.0,2927.0,156.0,157.0,39.0,3232.0,3332.0,24782.0,3307.0,4284.0,3326.0,3570.0,10920.0,659.0,1536.0
3,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,NaN,NaN,NaN,11.0,gmnakamuracyborg,GM,2785.0,United States,263.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,NaN,NaN,NaN,29.0,hikarunakamura,GM,2729.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46,1181 (43.18 %),None (34.52 %),610 (22.3 %),1096.0,enajer77,GM,NaN,Russia,2446.0,2530.0,18.0,12.0,15.0,2842.0,2842.0,397.0,62.0,323.0,2558.0,2576.0,1.0,0.0,2.0
112,97,"Saric, Ivan",2647.5,2631.0,2586.0,32,1153 (43.31 %),None (36.03 %),550 (20.66 %),101.0,dalmatinac101,GM,2660.0,Croatia,2712.0,2870.0,31.0,21.0,17.0,2820.0,3016.0,1992.0,656.0,1527.0,3098.0,3109.0,221.0,21.0,125.0
113,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42,NaN,NaN,NaN,117.0,vladimir_2020,GM,2650.0,Russia,2556.0,2785.0,49.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,99,"Liang, Awonder",2645.6,2518.4,2538.0,20,NaN,NaN,NaN,1379.0,rednova1729,GM,NaN,United States,2600.0,2765.0,33.0,5.0,9.0,3035.0,3042.0,563.0,165.0,397.0,3040.0,3082.0,1163.0,167.0,787.0


In [15]:
table = table.drop_duplicates()
table

,#,name,Classic,Rapid,Blitz,Age,Wins,Draws,Losses,Unnamed: 0,username,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32,2879 (48.53 %),718 (35.27 %),961 (16.2 %),3.0,magnuscarlsen,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31,2025 (42.75 %),547 (34.49 %),1078 (22.76 %),4.0,fabianocaruana,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,NaN,NaN,NaN,6.0,hikaru,GM,2814.0,United States,2781.0,2927.0,156.0,157.0,39.0,3232.0,3332.0,24782.0,3307.0,4284.0,3326.0,3570.0,10920.0,659.0,1536.0
3,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,NaN,NaN,NaN,11.0,gmnakamuracyborg,GM,2785.0,United States,263.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,NaN,NaN,NaN,29.0,hikarunakamura,GM,2729.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46,1181 (43.18 %),None (34.52 %),610 (22.3 %),1096.0,enajer77,GM,NaN,Russia,2446.0,2530.0,18.0,12.0,15.0,2842.0,2842.0,397.0,62.0,323.0,2558.0,2576.0,1.0,0.0,2.0
112,97,"Saric, Ivan",2647.5,2631.0,2586.0,32,1153 (43.31 %),None (36.03 %),550 (20.66 %),101.0,dalmatinac101,GM,2660.0,Croatia,2712.0,2870.0,31.0,21.0,17.0,2820.0,3016.0,1992.0,656.0,1527.0,3098.0,3109.0,221.0,21.0,125.0
113,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42,NaN,NaN,NaN,117.0,vladimir_2020,GM,2650.0,Russia,2556.0,2785.0,49.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,99,"Liang, Awonder",2645.6,2518.4,2538.0,20,NaN,NaN,NaN,1379.0,rednova1729,GM,NaN,United States,2600.0,2765.0,33.0,5.0,9.0,3035.0,3042.0,563.0,165.0,397.0,3040.0,3082.0,1163.0,167.0,787.0


In [9]:
table.to_csv(r'final_table.csv', index=False)

# nuevo escraping

In [10]:
top_100 = table['name'].tolist()

In [11]:
# Función para obtener las estadísticas de un jugador
def obtener_estadisticas(jugador):
    driver = webdriver.Chrome(PATH, options=opciones)
    driver.get(url)
    time.sleep(2)

    # Aceptar cookies
    aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')
    aceptar.click()
    
    # Variables de asignación predeterminadas
    wins = 'N/A'
    draws = 'N/A'
    losses = 'N/A'
    
    try:
        # Introducir el nombre del jugador
        texto = driver.find_element(By.XPATH, '//*[@id="data"]')
        texto.send_keys(jugador)
        texto.send_keys(Keys.ENTER)
        time.sleep(3)

        # Obtener las estadísticas
        wins = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
        draws = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
        losses = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text

        driver.quit()  # Cerrar el navegador
        
    except:
        time.sleep(1)

    return {'Jugador': jugador, 'Wins': wins, 'Draws': draws, 'Losses': losses}

# URL y opciones del driver
url = 'https://www.yottachess.com/'
# PATH ya definido
opciones = webdriver.ChromeOptions()


# Lista de jugadores top 100
# top_100 ya creada

# Realizar el procesamiento en paralelo para obtener las estadísticas de los jugadores
paralelo = Parallel(n_jobs=3,  verbose=True)
resultados = paralelo(delayed(obtener_estadisticas)(jugador) for jugador in tqdm(top_100))


# Crear el DataFrame con los resultados
data = pd.DataFrame(resultados)

# Mostrar el DataFrame
data

  0%|          | 0/116 [00:00<?, ?it/s]

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 13.6min
[Parallel(n_jobs=3)]: Done 116 out of 116 | elapsed: 34.5min finished


,Jugador,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
...,...,...,...,...
111,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
112,"Saric, Ivan",N/A,N/A,N/A
113,"Malakhov, Vladimir",N/A,N/A,N/A
114,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [13]:
data = data.drop_duplicates()
data

,Jugador,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
6,"Ding, Liren",760 (33.48 %),413 (47.4 %),434 (19.12 %)
7,"Firouzja, Alireza",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
...,...,...,...,...
111,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
112,"Saric, Ivan",N/A,N/A,N/A
113,"Malakhov, Vladimir",N/A,N/A,N/A
114,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)
